This will eventually be an introduction to numerical methods in partial differential equations. Over time, I hope to develop a resource which can be used to learn the theory and practice of numerical PDEs. The philosophy of this text is to be fully rigorous while connecting theory to practice with frequent coded examples of numerical solvers.

The essential prerequisites to be able to effectively use this text are calculus, linear algebra, and some coding knowledge. The first portion of this text is about finite difference methods, which are in effect carefully chosen Taylor approximations. Most other things can be picked up while reading.

The theory of numerical PDEs cannot exist apart from PDEs. Numerical schemes which work well for one type of PDE may fail miserably when adapted to even the simplest PDE of a different type. More importantly, knowledge of how the PDE can be solved and its solution informs the design of a numerical method to approximate its solution. PDEs with known exact solutions can be good test beds for numerical methods.

Let's start with an overview of some common PDEs. I make an attempt to use $t$ to represent a time-like variable and $x$ to represent a space-like variable. If there are three dimensions in particular, may use $\mathbf x = (x, y, z)$ I try to use a bold $\mathbf x$ to represent an $n$-dimensional space-like variable. I say time-like and space-like because there is mathematically no difference between the letters $x$ and $t$, but the nature of PDEs is such that the behavior of the solution may be qualitatively very different in the directions of different variables. The letters $u$ and $v$ commonly denote solutions to a PDE and are understood to be functions of $x$ and $t$. The partial derivative operators may be written as $\frac{\partial}{\partial x}$ or just $\partial_x$. The partial derivatives of $u$ are written as $u_x$ and $u_t$. Second derivatives are written similarly: $u_{tx} = (u_t)_x = \partial_x \partial_t u$. We freely assume that all solutions are sufficiently smooth that mixed partial derivatives commute, i.e., that $u_{xt} = u_{tx}$, etc. Note that there are [well-known examples](https://en.wikipedia.org/wiki/Symmetry_of_second_derivatives) in which this is not true.

The heat equation in one dimension: $u_t + au_{xx} = 0$, where $a > 0$ is a positive constant. The function $u$ models the evolution of temperature in a one-dimensional medium of constant thermal diffusivity $a$.

The heat equation in three dimensions: $u_t + a(u_{xx} + u_{yy} + u_{zz}) = 0$.

The wave equation: $u_{tt} = c^2(u_{xx} + u_{yy} + u_{zz})$.

The sum of all spacial second derivatives, $u_{xx} + u_{yy} + u_{zz}$, is common enough that it is given a dedicated symbol: $\Delta u$. So we can rewrite the heat equation as $u_t + a \Delta u = 0$ and the wave equation as $u_{tt} = c^2 \Delta u$. 

Poisson's equation: $\Delta u = f(\mathbf x)$, where $f$ is a function which may be specified. There is no time-like variable in this equation. If $f$ is just $0$, we get Laplace's equation: $\Delta u = 0$.

Black-Scholes equation: $v_t + \frac 1 2 \sigma^2 s^2 v_{ss} + r s v_s - r v = 0$, where $v$ is the price of a financial option on some stock, $s$ is the price of that stock, $r$ is the risk-free interest rate, and $\sigma$ is the volatility of the stock. This example emphasizes that PDEs need not model physical phenomena. 


The heat equation, the wave equation, and Laplace's equation all have significant physical importance and are archetypes for the three basic types of PDEs: parabolic, hyperbolic, and elliptic. Rather than define them, which takes some work, it is useful to describe them by their qualities. In parabolic equations, such as the heat equation, information travels with infinite speed. If our domain is the real interval $[-100, 100]$, then a change at the initial condition $u(-50, 0)$ for $x = -50$ will change the solution far away (say, $x = 50$) instantly: for any positive time $t > 0$, the solution $u(50, t)$ will be different than it would be without the change.

This is in stark contrast with the wave equation, in which information always travels with speed exactly $c$. If $c=1$, then a change at $x-=50$ can't be felt at $x=50$ until time $t = 100$. The behavior of information propagating at finite speed is the key feature of hyperbolic equations, such as the wave equation.

Lastly, elliptic PDEs (such as Poisson's equation and Laplace's equation) can be thought of as equations in which there is no time-like variable. They may represent the state of a system which is in equilibrium, i.e., not changing in time. This connection may be seen from the similarity between the heat equation and Laplace's equation. If we take the heat equation ($u_t + a \Delta u = 0$) and impose that the solution be in equilibrium, we get the condition that $u$ does not change in $t$, i.e., $u_t = 0$. If we make this substitution, we get the equation $a \Delta u = 0$, which is just Laplace's equation, and we see how in this case, the elliptic Laplace's equation may be thought of as a steady-state solution to the parabolic heat equation.